Загрузка и подготовка датасета EuroSAT

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

try:
    ds_train_full, ds_info = tfds.load(
        'eurosat',
        split='train',
        with_info=True,
        as_supervised=True
    )

    print("Датасет EuroSAT загружен успешно!")
    print("Количество классов:", ds_info.features['label'].num_classes)
    print("Размер изображений:", ds_info.features['image'].shape)
    print("Названия классов:", ds_info.features['label'].names)

except Exception as e:
    print("Ошибка при загрузке датасета EuroSAT:", e)
    print("Проверь, правильно ли установлены tensorflow и tensorflow_datasets.")


D:\University\bask_an_5\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Датасет EuroSAT загружен успешно!
Количество классов: 10
Размер изображений: (64, 64, 3)
Названия классов: ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']


Разделение данных на обучающую и тестовую выборки


In [2]:
class_names = ds_info.features['label'].names
total_examples = ds_info.splits['train'].num_examples
num_train_examples = int(0.7 * total_examples)

ds_train_full = ds_train_full.shuffle(buffer_size=10000, reshuffle_each_iteration=False)
ds_train = ds_train_full.take(num_train_examples)

ds_test = ds_train_full.skip(num_train_examples)


Предобработка изображений

In [3]:
NUM_CLASSES = ds_info.features['label'].num_classes
IMG_SIZE = 224
BATCH_SIZE = 32

def preprocess_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE), method='gaussian')
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return image, tf.one_hot(label, depth=NUM_CLASSES)

ds_train = ds_train.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

ds_train = ds_train.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


Создание и обучение полносвязной нейронной сети (Fully Connected NN)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Input

model = Sequential([
    Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

history = model.fit(ds_train, validation_data=ds_test, epochs=10)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 150528)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │       9,633,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,642,826 (36.78 MB)

 Trainable params: 9,642,826 (36.78 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
490/591 ━━━━━━━━━━━━━━━━━━━━ 12s 120ms/step - accuracy: 0.2116 - loss: 5.7350

In [ ]:
Визуализация результатов обучения



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# График потерь и точности на обучающей выборке
epochs = range(1, len(history.history['loss']) + 1)
loss = history.history['loss']
accuracy = history.history['accuracy']

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, loss, label='Loss')
plt.plot(epochs, accuracy, label='Accuracy')
plt.title('Потери и точность на обучающей выборке')
plt.xlabel('Эпоха')
plt.legend()

# График потерь и точности на валидационной выборке
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

plt.subplot(1, 2, 2)
plt.plot(epochs, val_loss, label='Val Loss')
plt.plot(epochs, val_accuracy, label='Val Accuracy')
plt.title('Потери и точность на валидационной выборке')
plt.xlabel('Эпоха')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
results = model.evaluate(ds_test)
print(f"Точность на тестовой выборке: {results[1]*100:.2f}%")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Подготовка данных для предсказаний
ds_test_np = ds_test.unbatch().batch(1)
x_test = np.concatenate([x.numpy() for x, y in ds_test_np], axis=0)
y_test = np.concatenate([y.numpy() for x, y in ds_test_np], axis=0)

# Получение предсказаний
predictions = model.predict(x_test)

# Визуализация 25 изображений с предсказаниями
plt.figure(figsize=(12, 12))
for idx in range(25):
    pred_idx = np.argmax(predictions[idx])
    true_idx = np.argmax(y_test[idx])

    plt.subplot(5, 5, idx + 1)
    plt.imshow((x_test[idx] + 1) / 2)  # Преобразование из [-1, 1] в [0, 1]
    color = 'green' if pred_idx == true_idx else 'red'
    plt.title(f"{class_names[pred_idx]} ({class_names[true_idx]})", color=color)
    plt.axis('off')

plt.tight_layout()
plt.show()


